In [1]:
# importing the packages necesary

import torch
from torch import nn
import pandas as pd
import numpy as np

## Model 0 - Prelims, using torch to build linear regression model



In [20]:
# building the model using torch

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        # linear function
        self.model = nn.Linear(input_dim, output_dim)


    def forward(self, X):
        out = self.model(X)
        return out

In [21]:
model = LinearRegressionModel(input_dim = 1,  output_dim = 1)

In [22]:
# setting the parameters
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

### example of a formula it's trying to learn

$$
y = f(x;\theta)
$$

where $f(.)$ is $3.4x_1 + 2.5$


In [23]:
x = torch.rand(1000,1)
y = x * 3.4 + 2.5

data = torch.cat((x,y), 1)

In [24]:
# prepare the dataset

batch_size = 10
n_epochs = 100

train_loader = torch.utils.data.DataLoader(dataset = data,
                                           batch_size = 10,
                                           shuffle = True)

In [25]:
for epochs in range(n_epochs):
    for i, _data in enumerate(train_loader):

        _x = _data.T[0].reshape(-1, 1)
        _y = _data.T[-1].reshape(-1, 1)

        # zero the gradients
        optimizer.zero_grad()

        outputs = model(_x)
        loss = loss_fn(outputs, _y)

        loss.backward()

        optimizer.step()


In [ ]:
model.state_dict()

OrderedDict([('model.weight', tensor([[3.4000]])),
             ('model.bias', tensor([2.5000]))])

In [28]:
_x.shape

torch.Size([10, 1])

## Model 1 - Feedforward neural network

In [82]:
class FeedforwardModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)


    def forward(self, X):
        out = self.fc1(X)
        out = self.relu(out)
        out = self.fc2(out)

        return out

### example problem - XOR problem

Input - list of tuple of 1 or 0
Output - if it's exactly 1, it will return 1

$$A \oplus B = (A \land \lnot B) \lor (\lnot A \land B) $$

In [83]:
X = torch.tensor([
    (0,0),
    (1,0),
    (1,1),
    (0,1)
], dtype=torch.float32)

y = torch.tensor([
    0, 1, 0, 1
], dtype=torch.float32)
y = y.reshape(len(y), 1)

In [84]:
model = FeedforwardModel(input_dim = 2, hidden_dim = 2, output_dim = 1)

In [85]:
# example of using mse loss

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [86]:
n_epochs = 1000

for epoch in range(n_epochs):

    y_pred = model(X)
    loss = loss_fn(y_pred, y)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

In [87]:
model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.8792, -0.8799],
                      [-0.9175,  0.9181]])),
             ('fc1.bias', tensor([ 8.7142e-02, -4.9837e-07])),
             ('fc2.weight', tensor([[1.1374, 1.1971]])),
             ('fc2.bias', tensor([-0.0991]))])

In [88]:
(y_pred > 0.5).float()

tensor([[0.],
        [1.],
        [0.],
        [1.]])